In [262]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt 
import pandas as pd
import scipy as sp
from scipy import stats
from scipy.stats import linregress
import glob
import os #Imports operating system api

path = r'C:\Users\percuoco\Desktop\IODPDataMining_USEME\SRM_new' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    if ("SRM" in filename and not("SRMD" in filename)):
            df = pd.read_csv(filename, index_col=None, header=0, error_bad_lines = False, low_memory=False)    
            li.append(df)
            
dataFrame = pd.concat(li, axis=0, ignore_index=True)
dataFrame.head(10);

In [263]:
#Drops the NA and NaN values from the selected columns

columns_to_search = [
#'Exp',
#'Site',
#'Hole',
#'Core',
#'Type',
#'#Sect',
#'A.W',
'Offset..cm.',
'Depth.CSF.A..m.',
'Depth.CSF.B..m.',
'Treatment.Type',
'Treatment.Value',
'Inclination.background...drift.corrected..deg.',
'Declination.background...drift.corrected..deg.',
'Intensity.background...drift.corrected..A.m.',
'Inclination.raw..deg.',
'Declination.raw..deg.',
'Intensity.raw..A.m.',
'Magnetic.moment.x..Am..',
'Magnetic.moment.y..Am..',
'Magnetic.moment.z..Am..',
'Sample.orientation',
#'Raw.data',
#'Timestamp..UTC.',
#'Instrument',
'Text.ID',
'Test.No.',
#'Comments',
#'matched_litho_prefix',
#'matched_litho_principal',
#'matched_litho_suffix',
#'longitude_DD',
#'latitude_DD',
#'water_depth_m'
]
dataFrame.dropna(subset=columns_to_search);

In [264]:
#Running into the issue where I cannot change datatypes otherwise the read_csv operation fails:
#I narrowed it down to values of '1.1380E 0', and '1.9441E 0' in the 'Intensity.raw..A.m.' column
#So I am dropping those here

# Get names of indexes where the raw intensities have errors:
#values_to_drop = ["1.1380E 0","1.9441E 0", "NA", "NaN"]

#indexNames = dataFrame[dataFrame[columns_to_search].isin(values_to_drop)].index
#print(indexNames)
#dataFrame.drop(indexNames)


In [265]:
#Convert the datatypes where I can:
dataFrame.astype({
'Exp': str,
'Site': str,
'Hole': str,
'Core': str,
'Type': str,
'Sect': str,
'A.W': str,
'Offset..cm.': np.float64,
'Depth.CSF.A..m.': np.float64,
'Depth.CSF.B..m.': np.float64,
'Treatment.Type': str,
'Treatment.Value': int,
'Inclination.background...drift.corrected..deg.': np.float64,
'Declination.background...drift.corrected..deg.': np.float64,
'Intensity.background...drift.corrected..A.m.': np.float64,
'Inclination.raw..deg.': np.float64,
'Declination.raw..deg.': np.float64,
'Intensity.raw..A.m.': np.float64,
'Magnetic.moment.x..Am..': np.float64,
'Magnetic.moment.y..Am..': np.float64,
'Magnetic.moment.z..Am..': np.float64,
'Sample.orientation': str,
'Raw.data': str,
'Timestamp..UTC.': str,
'Instrument': str,
'Text.ID': str,
'Test.No.': str,
'Comments': str,
'matched_litho_prefix': str,
'matched_litho_principal': str,
'matched_litho_suffix': str,
'longitude_DD': np.float64,
'latitude_DD': np.float64,
'water_depth_m': np.float64}).dtypes;                    

In [266]:
#Show information about the dataframe and datatypes for final verification

#dataFrame.info()
#dataFrame.dtypes

In [267]:
#Get unique values for lithos, prefices, and suffices. Need to remove NaN values from columns

principals = list((dataFrame['matched_litho_principal']).unique());
principals = ['All'] + list(filter(lambda x: str(x) != 'nan', principals))

prefices = list((dataFrame['matched_litho_prefix']).unique())
prefices = ['All'] + list(filter(lambda x: str(x) != 'nan', prefices))

suffices = list((dataFrame['matched_litho_suffix']).unique())
suffices = ['All'] + list(filter(lambda x: str(x) != 'nan', suffices))


# **Below is a listbox for filtering the histograms by principal lithology**

In [268]:
#This is one method for generating in interactive widget. The output widget depends on the parameter datatype
#I'm passing a list of strings, so it autogenerates a list dropdown
#The selected value can be accessed with q.result

from ipywidgets import widgets, interactive, Output

from IPython.display import display

#def ident(lithology):
#    return lithology

#q = interactive(ident, lithology=lithos)
#display(q) #To display the widget
out = Output()

In [269]:
column_selector = widgets.SelectMultiple(
    options=dataFrame.columns,
    #rows=10,
    description='Columns to show',
    disabled=False
)

In [270]:
#This is another method for generating in interactive widget. The output widget depends on the parameter datatype
#I'm passing a list of strings, so it autogenerates a listdropdown
#The selected value can be accessed with s.value
dd_principal = widgets.Dropdown(options=principals, description='Principal')
dd_prefix = widgets.Dropdown(options=prefices, description='Prefix')
dd_suffix = widgets.Dropdown(options=suffices, description='Suffix')

In [271]:
#This should return a single dataframe, using three parameters to filter.
def query(principal, prefix, suffix):
    out.clear_output()
    principal_filter = dataFrame if principal == 'All' else dataFrame[dataFrame['matched_litho_principal'] == principal]
    prefix_filter = principal_filter if prefix == 'All' else principal_filter[principal_filter['matched_litho_prefix'] == prefix]
    suffix_filter = prefix_filter if suffix == 'All' else prefix_filter[prefix_filter['matched_litho_suffix'] == suffix]
    with out:
        display(suffix_filter[list(column_selector.value)])

In [272]:
def dd_principal_observer(bunch):
    query(bunch.new, dd_prefix.value, dd_suffix.value)

def dd_prefix_observer(bunch):
    query(dd_principal.value, bunch.new, dd_suffix.value)

def dd_suffix_observer(bunch):
    query(dd_principal.value, dd_prefix.value,  bunch.new)


In [273]:
dd_principal.observe(dd_principal_observer, names='value')
dd_prefix.observe(dd_prefix_observer, names='value')
dd_suffix.observe(dd_suffix_observer, names='value')

In [274]:
vbox1 = widgets.VBox([dd_principal, dd_prefix, dd_suffix])
tab = widgets.Tab()
tab.children = [vbox1, column_selector]
tab.set_title(0, 'Lithology Filters')
tab.set_title(1, 'Display Columns')
display(tab)
display(out)

Output()

In [275]:
#display(out)

In [276]:
#After changing the widget make sure to SHFT+ENT to rerun this cell
def histogram_plots(bunch):
    out.clear_output()
    #suppresses the plots
    plt.ioff()
    x = dataFrame[dataFrame['matched_litho_principal'] == bunch.new]
    a = x[x['Treatment.Value']==0] 
    a= a['Intensity.background...drift.corrected..A.m.'];

    b = x[x['Treatment.Value']==10] 
    b= b['Intensity.background...drift.corrected..A.m.'];

    c = x[x['Treatment.Value']==20] 
    c= c['Intensity.background...drift.corrected..A.m.'];

    n_bins_intensity = 20
    n_bins = 20

    fig, axs = plt.subplots(1, 3, sharey=False, tight_layout=True)


    # We can set the number of bins with the `bins` kwarg
    axs[0].hist(a, bins=n_bins_intensity)
    axs[0].set(ylim=(0, 1000))
    axs[1].hist(b, bins=n_bins_intensity)
    axs[2].hist(c, bins=n_bins_intensity);

    axs[1].set_xlabel('Intensity (A/m)', fontname='Arial', fontsize=12)
    axs[0].set_ylabel('Frequency', fontname='Arial', fontsize=12);

    axs[0].set_title('Demag Lvl: 0 mT')
    axs[1].set_title('Demag Lvl: 10 mT')
    axs[2].set_title('Demag Lvl: 20 mT');
    with out:
        display()
        

def registry(bunch):
    out.clear_output()
    histogram_plots(bunch)
    dd_principal_observer(bunch)
    with out:
        display()
    
dd_principal.observe(registry, names='value')


In [277]:
def highlighter(myvalue):
    if isinstance(myvalue, float):
        return 'color: {0}'.format('green' if myvalue <= 10 else 'black')
    return 'color: black'

In [278]:
x = dataFrame.head(10)
x.columns[7]
x.style.applymap(highlighter, subset=x.columns[14])


,Exp,Site,Hole,Core,Type,Sect,A.W,Offset..cm.,Depth.CSF.A..m.,Depth.CSF.B..m.,Treatment.Type,Treatment.Value,Inclination.background...drift.corrected..deg.,Declination.background...drift.corrected..deg.,Intensity.background...drift.corrected..A.m.,Inclination.raw..deg.,Declination.raw..deg.,Intensity.raw..A.m.,Magnetic.moment.x..Am..,Magnetic.moment.y..Am..,Magnetic.moment.z..Am..,Sample.orientation,Raw.data,Timestamp..UTC.,Instrument,Text.ID,Test.No.,Comments,matched_litho_prefix,matched_litho_principal,matched_litho_suffix,longitude_DD,latitude_DD,water_depth_m
0,362,U1480,A,1,H,1,A,0.000000,0.000000,0.000000,NRM,0,66.000000,189.800000,0.238550,66.000000,189.700000,0.238430,-0.000010,-0.000002,0.000038,SECTION HALF : ARCHIVE,35389801.000000,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,ash/tuff,nan,91.605802,3.034100,4147.530000
1,362,U1480,A,1,H,1,A,2.500000,0.025000,0.025000,NRM,0,68.900000,195.400000,0.170080,68.900000,195.300000,0.169960,-0.000006,-0.000002,0.000028,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,ash/tuff,nan,91.605802,3.034100,4147.530000
2,362,U1480,A,1,H,1,A,5.000000,0.050000,0.050000,NRM,0,80.400000,197.400000,0.107770,80.400000,197.300000,0.107660,-0.000002,-0.000001,0.000018,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,ash/tuff,nan,91.605802,3.034100,4147.530000
3,362,U1480,A,1,H,1,A,7.500000,0.075000,0.075000,NRM,0,88.400000,9.500000,0.076116,88.400000,10.600000,0.076005,0.000000,0.000000,0.000013,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,clay,nan,91.605802,3.034100,4147.530000
4,362,U1480,A,1,H,1,A,10.000000,0.100000,0.100000,NRM,0,84.400000,17.200000,0.060200,84.400000,17.700000,0.060091,0.000001,0.000000,0.000010,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,clay,nan,91.605802,3.034100,4147.530000
5,362,U1480,A,1,H,1,A,12.500000,0.125000,0.125000,NRM,0,85.700000,20.800000,0.051611,85.600000,21.400000,0.051502,0.000000,0.000000,0.000009,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,clay,nan,91.605802,3.034100,4147.530000
6,362,U1480,A,1,H,1,A,15.000000,0.150000,0.150000,NRM,0,87.200000,32.300000,0.047056,87.200000,33.000000,0.046947,0.000000,0.000000,0.000008,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,clay,nan,91.605802,3.034100,4147.530000
7,362,U1480,A,1,H,1,A,17.500000,0.175000,0.175000,NRM,0,87.800000,51.600000,0.045057,87.800000,52.200000,0.044947,0.000000,0.000000,0.000008,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,clay,nan,91.605802,3.034100,4147.530000
8,362,U1480,A,1,H,1,A,20.000000,0.200000,0.200000,NRM,0,88.100000,70.400000,0.044596,88.000000,70.700000,0.044485,0.000000,0.000000,0.000008,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,clay,nan,91.605802,3.034100,4147.530000
9,362,U1480,A,1,H,1,A,22.500000,0.225000,0.225000,NRM,0,87.800000,63.800000,0.045278,87.700000,64.200000,0.045169,0.000000,0.000000,0.000008,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,clay,nan,91.605802,3.034100,4147.530000


In [279]:
x = dataFrame.head(10)
x.style.applymap(highlighter, subset=x.columns[14])


,Exp,Site,Hole,Core,Type,Sect,A.W,Offset..cm.,Depth.CSF.A..m.,Depth.CSF.B..m.,Treatment.Type,Treatment.Value,Inclination.background...drift.corrected..deg.,Declination.background...drift.corrected..deg.,Intensity.background...drift.corrected..A.m.,Inclination.raw..deg.,Declination.raw..deg.,Intensity.raw..A.m.,Magnetic.moment.x..Am..,Magnetic.moment.y..Am..,Magnetic.moment.z..Am..,Sample.orientation,Raw.data,Timestamp..UTC.,Instrument,Text.ID,Test.No.,Comments,matched_litho_prefix,matched_litho_principal,matched_litho_suffix,longitude_DD,latitude_DD,water_depth_m
0,362,U1480,A,1,H,1,A,0.000000,0.000000,0.000000,NRM,0,66.000000,189.800000,0.238550,66.000000,189.700000,0.238430,-0.000010,-0.000002,0.000038,SECTION HALF : ARCHIVE,35389801.000000,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,ash/tuff,nan,91.605802,3.034100,4147.530000
1,362,U1480,A,1,H,1,A,2.500000,0.025000,0.025000,NRM,0,68.900000,195.400000,0.170080,68.900000,195.300000,0.169960,-0.000006,-0.000002,0.000028,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,ash/tuff,nan,91.605802,3.034100,4147.530000
2,362,U1480,A,1,H,1,A,5.000000,0.050000,0.050000,NRM,0,80.400000,197.400000,0.107770,80.400000,197.300000,0.107660,-0.000002,-0.000001,0.000018,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,ash/tuff,nan,91.605802,3.034100,4147.530000
3,362,U1480,A,1,H,1,A,7.500000,0.075000,0.075000,NRM,0,88.400000,9.500000,0.076116,88.400000,10.600000,0.076005,0.000000,0.000000,0.000013,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,clay,nan,91.605802,3.034100,4147.530000
4,362,U1480,A,1,H,1,A,10.000000,0.100000,0.100000,NRM,0,84.400000,17.200000,0.060200,84.400000,17.700000,0.060091,0.000001,0.000000,0.000010,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,clay,nan,91.605802,3.034100,4147.530000
5,362,U1480,A,1,H,1,A,12.500000,0.125000,0.125000,NRM,0,85.700000,20.800000,0.051611,85.600000,21.400000,0.051502,0.000000,0.000000,0.000009,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,clay,nan,91.605802,3.034100,4147.530000
6,362,U1480,A,1,H,1,A,15.000000,0.150000,0.150000,NRM,0,87.200000,32.300000,0.047056,87.200000,33.000000,0.046947,0.000000,0.000000,0.000008,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,clay,nan,91.605802,3.034100,4147.530000
7,362,U1480,A,1,H,1,A,17.500000,0.175000,0.175000,NRM,0,87.800000,51.600000,0.045057,87.800000,52.200000,0.044947,0.000000,0.000000,0.000008,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,clay,nan,91.605802,3.034100,4147.530000
8,362,U1480,A,1,H,1,A,20.000000,0.200000,0.200000,NRM,0,88.100000,70.400000,0.044596,88.000000,70.700000,0.044485,0.000000,0.000000,0.000008,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,clay,nan,91.605802,3.034100,4147.530000
9,362,U1480,A,1,H,1,A,22.500000,0.225000,0.225000,NRM,0,87.800000,63.800000,0.045278,87.700000,64.200000,0.045169,0.000000,0.000000,0.000008,nan,nan,2016-08-13 01:49:56,SRM,SHLF7851761,99542811,nan,nan,clay,nan,91.605802,3.034100,4147.530000


In [280]:
%lsmagic -gui?


In [ ]:
%lsmagic -gui

In [281]:
%lsmagic -gui

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python 

In [282]:
%lsmagic -gui

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python 

In [283]:
%lsmagic -gui

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python 

In [284]:
%lsmagic -gui

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python 

In [285]:
%lsmagic -gui

Available line magics:
%alias  %alias_magic  %autoawait  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %conda  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %pip  %popd  %pprint  %precision  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%markdown  %%perl  %%prun  %%pypy  %%python 

%lsmagic **-gui**

## Holy crap you can even add latex in this badboy!
$e^{i\pi} + 1 = 0$